<h2> Exercici de Recuperació 1 </h2>
<br>- Omnirobot de 3 rodes. 


### 1. Connexió al coppelia


In [1]:
import sim
import time
import math
import numpy as np
def connect(port):
# Establece la conexión a COPPELIA
# El port debe coincidir con el puerto de conexión en VREP  -- DALE AL PLAY !!!
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID


In [4]:
# Conectarse al servidor de COPPELIA
# *** _Hay que ejecutarlo cada vez que se reinicia la simulación ***
clientID = connect(19999)

conectado a 19999


<h3> 2. Controladors</h3>

In [5]:
# Obtenemos los controladores para cada eje de las ruedas y el ultrasonido
ret,WCR=sim.simxGetObjectHandle(clientID,'OmniWCR',sim.simx_opmode_blocking)
ret,WCL=sim.simxGetObjectHandle(clientID,'OmniWCL',sim.simx_opmode_blocking)
ret,WC=sim.simxGetObjectHandle(clientID,'OmniWC',sim.simx_opmode_blocking)

# Obtenim també "omniRob" que serà molt util per veure la posició del robot 
ret, omniRob = sim.simxGetObjectHandle(clientID, 'omniRob', sim.simx_opmode_blocking)


print(WCR, WCL, WC, omniRob)

21 26 16 15


<h3> 3. Moviments del robot</h3>
<br>Heu de realitzar les següents funcions per realitzar moviments lineals i figures :
<br> 1- Funció per avançar en un determinat temps
<br> 2- Funció per girar 90graus a la dreta (mireu la propietat d'un objecte)
<br> 3- Funció per girar 90 graus a l'esquerra (mireu la propietat d'un objecte)
<br> 4- Funció per realitzar un quadrat
<br> 5- Funció per realitzar un triangle
<br> 6- Funció per realitzar un cercle




In [18]:

#exemple de possibles moviments
velocidad = 1
time.sleep(2)
#Avanzar hacia delante durante 3 segundos:
sim.simxSetJointTargetVelocity(clientID, WCR,velocidad,sim.simx_opmode_streaming)
sim.simxSetJointTargetVelocity(clientID, WCL,-velocidad,sim.simx_opmode_streaming)
time.sleep(3)
#Retroceder hacia atras durante 3 segundos:
sim.simxSetJointTargetVelocity(clientID, WCR,-velocidad,sim.simx_opmode_streaming)
sim.simxSetJointTargetVelocity(clientID, WCL,-velocidad,sim.simx_opmode_streaming)
time.sleep(3)
#Girar hacia la derecha durante 3 segundos:
sim.simxSetJointTargetVelocity(clientID, WCR,-velocidad,sim.simx_opmode_streaming)
sim.simxSetJointTargetVelocity(clientID, WCL,velocidad,sim.simx_opmode_streaming)
time.sleep(3)
#Parar:
sim.simxSetJointTargetVelocity(clientID, WCR,0,sim.simx_opmode_streaming)
sim.simxSetJointTargetVelocity(clientID, WCL,0,sim.simx_opmode_streaming)
time.sleep(3)

<h3> 4 Incorporació de sensors de distancia </h3>

Heu d'utilitzar 4 sensors i colocar a cada costat 1 sensor. Intenteu realitzar un programa per moure el robot dins d'un escenari amb  obstacles ( poden ser parets, objectes del coppelia )  i intentar que el robot detecti els obstacles i els pugui evitar 


<h3> Podeu veure els videos de l'exercici 10 on s'explica com afegir aquests sensors </h3>








In [4]:
# 1- Funció per avançar en un determinat temps

def goForward(clientID, wheel_L, wheel_R, duration):
    speed = 1 # No la passo per parametre perquè l'enunciat només especifica "en un determinat temps"

    # Moviment als controladors especificats
    sim.simxSetJointTargetVelocity(clientID, wheel_L, speed, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_R, -speed, sim.simx_opmode_streaming)

    # Esperar durant el temps definit a "duration"
    time.sleep(duration)

    # Parar després d'esperar
    sim.simxSetJointTargetVelocity(clientID, wheel_L, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_R, 0, sim.simx_opmode_streaming)

# Exemple d'us amb els controladors WCR, WCL:
duration = 3  # Duració (segons)
goForward(clientID, WCR, WCL, duration)

In [6]:
# 2- Funció per girar 90graus a la dreta (mireu la propietat d'un objecte)

def turnRight(clientID, omniRob, wheel_1, wheel_2, wheel_3):
    speed = 0.5  # Velocitat de gir lenta per a que s'identifiqui correctament el gir

    # Obtenir l'orientació inicial del robot
    ret, orientacio_inicial = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
    
    # print("orientacio_inicial: ", orientacio_inicial)

    if ret != sim.simx_return_ok:
        print("Error al obtenir l'orientació inicial")
        return
    
    # Calcular l'orientació objectiu (90 graus a la dreta)
    angulo_objetivo = orientacio_inicial[2] - (90 * (math.pi / 180))  # Graus a radiants

    # print("angulo_objetivo: ", angulo_objetivo)
    
    # Ajustar l'angle objectiu perquè estigui entre -pi i pi]
    if angulo_objetivo < -math.pi:
        angulo_objetivo += 2 * math.pi
    
    # Establir velocitats de les rodes per girar a la dreta
    sim.simxSetJointTargetVelocity(clientID, wheel_1, speed, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_2, speed, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_3, speed, sim.simx_opmode_streaming)
    
    # Bucle fins que el robot gira 90 graus a la dreta
    while True:
        ret, orientacion_actual = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
        
        if ret != sim.simx_return_ok:
            print("Error al obtenir l'orientació actual")
            break
        
        angulo_actual = orientacion_actual[2]
        
        # print("[While] angulo_actual: ", angulo_actual)
        # print("[While] dif: ", abs(angulo_actual - angulo_objetivo))

        if abs(angulo_actual - angulo_objetivo) < 0.15: # Quan estem arribant, disminuim velocitat per prioritzar precisió
            speed = 0.01
            # Establir nova velocitats de les rodes per girar a la dreta
            sim.simxSetJointTargetVelocity(clientID, wheel_1, speed, sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, wheel_2, speed, sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, wheel_3, speed, sim.simx_opmode_streaming)


        if abs(angulo_actual - angulo_objetivo) < 0.005: # Diferència entre angle objectiu i actual; 0.01 es la "precisió"
            break
    
    # Parar després de girar
    sim.simxSetJointTargetVelocity(clientID, wheel_1, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_2, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_3, 0, sim.simx_opmode_streaming)

# Exemple d'ús de la funció

turnRight(clientID, omniRob, WCR, WCL, WC)


Error al obtenir l'orientació actual


In [27]:
# 3- Funció per girar 90 graus a l'esquerra (mireu la propietat d'un objecte)

def turnLeft(clientID, omniRob, wheel_1, wheel_2, wheel_3):
    speed = 0.4  # Velocitat de gir lenta per a que s'identifiqui correctament el gir

    # Obtenir l'orientació inicial del robot
    ret, orientacio_inicial = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
    
    # print("orientacio_inicial: ", orientacio_inicial)

    if ret != sim.simx_return_ok:
        print("Error al obtenir l'orientació inicial")
        return
    
    # Calcular l'orientació objectiu (90 graus a la ESQUERRA = 360-90 = 270 graus)
    angulo_objetivo = orientacio_inicial[2] - (270 * (math.pi / 180))  # Graus a radiants

    # print("angulo_objetivo: ", angulo_objetivo)
    
    # Ajustar l'angle objectiu perquè estigui entre -pi i pi]
    if angulo_objetivo < -math.pi:
        angulo_objetivo += 2 * math.pi
    
    # Establir velocitats de les rodes per girar a la ESQUERRA
    sim.simxSetJointTargetVelocity(clientID, wheel_1, -speed, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_2, -speed, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_3, -speed, sim.simx_opmode_streaming)
    
    # Bucle fins que el robot gira 90 graus a la ESQUERRA
    while True:
        ret, orientacion_actual = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
        
        if ret != sim.simx_return_ok:
            print("Error al obtenir l'orientació actual")
            break
        
        angulo_actual = orientacion_actual[2]
        
        # print("[While] angulo_actual: ", angulo_actual)
        # print("[While] dif: ", abs(angulo_actual - angulo_objetivo))

        if abs(angulo_actual - angulo_objetivo) < 0.15: # Quan estem arribant, disminuim velocitat per prioritzar precisió
            speed = 0.01
            # Establir nova velocitats de les rodes per girar a la dreta
            sim.simxSetJointTargetVelocity(clientID, wheel_1, -speed, sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, wheel_2, -speed, sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, wheel_3, -speed, sim.simx_opmode_streaming)

        if abs(angulo_actual - angulo_objetivo) < 0.005: # Diferència entre angle objectiu i actual; 0.01 es la "precisió"
            break
    
    # Parar després de girar
    sim.simxSetJointTargetVelocity(clientID, wheel_1, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_2, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_3, 0, sim.simx_opmode_streaming)

# Exemple d'ús de la funció

turnLeft(clientID, omniRob, WCR, WCL, WC)


In [31]:
# 4- Funció per realitzar un quadrat

# Aprofitant les funcions anterior, podem fer un quadrat fàcilment

def square(clientID, omniRob, wheel_1, wheel_2, wheel_3):
    for _ in range(4):
        goForward(clientID, wheel_1, wheel_2, 3)
        turnRight(clientID, omniRob, wheel_1, wheel_2, wheel_3)

square(clientID, omniRob, WCR, WCL, WC)

In [8]:
# 5- Funció per realitzar un triangle

# Nomes podrem aprofitar la funció "goForward", ja que les altres giren 90 graus, i ara necesitarem que giri 120 en cada gir (360/3 = 120)
# Per simplificar, fare una funció "turn120degrees".

def turn120degrees(clientID, omniRob, wheel_1, wheel_2, wheel_3):
    speed = 0.4  # Velocitat de gir lenta per a que s'identifiqui correctament el gir

    # Obtenir l'orientació inicial del robot
    ret, orientacio_inicial = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
    
    # print("orientacio_inicial: ", orientacio_inicial)

    if ret != sim.simx_return_ok:
        print("Error al obtenir l'orientació inicial")
        return
    
    # Calcular l'orientació objectiu (120 graus a la DRETA = 360/3 = 120 graus)
    angulo_objetivo = orientacio_inicial[2] - (120 * (3.14159265 / 180))  # Graus a radiants

    # print("angulo_objetivo: ", angulo_objetivo)
    
    # Ajustar l'angle objectiu perquè estigui entre -pi i pi]
    if angulo_objetivo < -3.14159265:
        angulo_objetivo += 2 * 3.14159265
    
    # Establir velocitats de les rodes per girar a la DRETA
    sim.simxSetJointTargetVelocity(clientID, wheel_1, speed, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_2, speed, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_3, speed, sim.simx_opmode_streaming)
    
    # Bucle fins que el robot gira 120 graus a la DRETA
    while True:
        ret, orientacion_actual = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
        
        if ret != sim.simx_return_ok:
            print("Error al obtenir l'orientació actual")
            break
        
        angulo_actual = orientacion_actual[2]
        
        # print("[While] angulo_actual: ", angulo_actual)
        # print("[While] dif: ", abs(angulo_actual - angulo_objetivo))

        if abs(angulo_actual - angulo_objetivo) < 0.15: # Quan estem arribant, disminuim velocitat per prioritzar precisió
            speed = 0.05
            # Establir nova velocitats de les rodes per girar a la dreta
            sim.simxSetJointTargetVelocity(clientID, wheel_1, speed, sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, wheel_2, speed, sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, wheel_3, speed, sim.simx_opmode_streaming)

        if abs(angulo_actual - angulo_objetivo) < 0.005: # Diferència entre angle objectiu i actual; 0.01 es la "precisió"
            break
    
    # Parar després de girar
    sim.simxSetJointTargetVelocity(clientID, wheel_1, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_2, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_3, 0, sim.simx_opmode_streaming)

def triangle(clientID, omniRob, wheel_1, wheel_2, wheel_3):
    for _ in range(3):
        goForward(clientID, wheel_1, wheel_2, 3)
        turn120degrees(clientID, omniRob, wheel_1, wheel_2, wheel_3)

ret, omniRob = sim.simxGetObjectHandle(clientID, 'omniRob', sim.simx_opmode_blocking)

triangle(clientID, omniRob, WCR, WCL, WC)

In [20]:
# 6- Funció per realitzar un cercle

# Per fer aquesta funció, haurem de definir velocitats concretes per cada controlador de les rodes.


3.141592653589793


conectado a 19999
Conectado a CoppeliaSim


TypeError: get_wheel_distance() missing 1 required positional argument: 'wheel2_handle'

In [ ]:
# 4 Incorporació de sensors de distancia
# Heu d'utilitzar 4 sensors i colocar a cada costat 1 sensor. Intenteu realitzar un programa per moure el robot dins d'un escenari 
# amb  obstacles ( poden ser parets, objectes del coppelia )  i intentar que el robot detecti els obstacles i els pugui evitar 